In [9]:
# Required packages
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras import optimizers

In [10]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [11]:
# load data
dataset = np.loadtxt("data_tp1", delimiter=",")
X_train = dataset[:, 1:].reshape(dataset.shape[0],1,28, 28).astype( 'float32' )
y_train = dataset[:,0]
X_test = dataset[:,1:].reshape(dataset.shape[0],1, 28, 28).astype( 'float32' )
y_test = dataset[:,0]

In [12]:
# flatten 28*28 images to a 784 vector for each image
num_pixels = 784
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

In [13]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

In [14]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [15]:
# define model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(25, input_dim=25, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    # Set the gradient
    sgd = optimizers.SGD(lr=.5)
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [16]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Training Error: %.2f%%" % (100-scores[1]*100))

Train on 5000 samples, validate on 5000 samples
Epoch 1/10
 - 1s - loss: 1.8167 - acc: 0.3986 - val_loss: 1.5082 - val_acc: 0.5260
Epoch 2/10
 - 1s - loss: 0.8739 - acc: 0.7224 - val_loss: 1.0055 - val_acc: 0.6604
Epoch 3/10
 - 1s - loss: 0.5909 - acc: 0.8122 - val_loss: 0.4035 - val_acc: 0.8708
Epoch 4/10
 - 1s - loss: 0.3793 - acc: 0.8760 - val_loss: 0.2329 - val_acc: 0.9328
Epoch 5/10
 - 1s - loss: 0.2525 - acc: 0.9158 - val_loss: 0.1981 - val_acc: 0.9378
Epoch 6/10
 - 1s - loss: 0.1857 - acc: 0.9396 - val_loss: 0.1326 - val_acc: 0.9614
Epoch 7/10
 - 1s - loss: 0.1398 - acc: 0.9574 - val_loss: 0.1155 - val_acc: 0.9686
Epoch 8/10
 - 1s - loss: 0.1084 - acc: 0.9682 - val_loss: 0.0787 - val_acc: 0.9796
Epoch 9/10
 - 1s - loss: 0.0900 - acc: 0.9754 - val_loss: 0.0576 - val_acc: 0.9864
Epoch 10/10
 - 1s - loss: 0.0603 - acc: 0.9854 - val_loss: 0.0485 - val_acc: 0.9896
Training Error: 1.04%
